In [1]:
import nltk
from nltk.tokenize import sent_tokenize
import spacy
from nltk.stem import WordNetLemmatizer

In [2]:
nlp = spacy.load('en_core_web_sm')
lemmatizer = WordNetLemmatizer()
textfile = "data/set4/a7.txt"
#put entire text file into a list of sentences
def get_text(textFile):
	text = []
	with open(textFile, "r") as f:
		for line in f:
			text = text + sent_tokenize(line)
	return text
text = get_text(textfile)


In [11]:
print(pos_tag_lst("hello my name is Laura"))

{0: [('h', 'X', 'LS', 'ROOT', False)], 1: [('e', 'VERB', 'VB', 'ROOT', False)], 2: [('l', 'NOUN', 'NN', 'ROOT', False)], 3: [('l', 'NOUN', 'NN', 'ROOT', False)], 4: [('o', 'INTJ', 'UH', 'ROOT', False)], 5: [(' ', 'SPACE', '_SP', 'ROOT', False)], 6: [('m', 'VERB', 'VBP', 'ROOT', False)], 7: [('y', 'PROPN', 'NNP', 'ROOT', False)], 8: [(' ', 'SPACE', '_SP', 'ROOT', False)], 9: [('n', 'ADV', 'RB', 'ROOT', False)], 10: [('a', 'DET', 'DT', 'ROOT', True)], 11: [('m', 'VERB', 'VBP', 'ROOT', False)], 12: [('e', 'VERB', 'VB', 'ROOT', False)], 13: [(' ', 'SPACE', '_SP', 'ROOT', False)], 14: [('i', 'PRON', 'PRP', 'ROOT', True)], 15: [('s', 'VERB', 'VBZ', 'ROOT', False)], 16: [(' ', 'SPACE', '_SP', 'ROOT', False)], 17: [('L', 'NOUN', 'NN', 'ROOT', False)], 18: [('a', 'DET', 'DT', 'ROOT', True)], 19: [('u', 'NOUN', 'NN', 'ROOT', False)], 20: [('r', 'NOUN', 'NN', 'ROOT', False)], 21: [('a', 'DET', 'DT', 'ROOT', True)]}


In [3]:
def pos_tag_lst(text):
	#list of sentences
	POS_tag_dict = dict()
	for i,line in enumerate(text):
		tags = []
		doc = nlp(str(line))
        print(doc)
		for token in doc:
#             print(doc)
#             print(token)
			tags.append((token.text, token.pos_, token.tag_, token.dep_, token.is_stop))
		if len(tags) != 0:
			POS_tag_dict[i] = tagsb
	return POS_tag_dict

def pos_tag_sentence(sentence):
	#list of sentences
	POS_tag_dict = dict()
	text = sentence.split()
	for line in text:
		tags = []
		doc = nlp(str(line))
		for token in doc:
			tags.append((token.pos_, token.tag_, token.dep_, token.is_stop, ))
		if len(tags) != 0:
			POS_tag_dict[token.text] = tags[0]
	return POS_tag_dict

#Token dict
def dependency_dict(doc):
	out = dict()
	root = ''
	for token in doc:
		out[token.text] = (token.dep_, token.head.text, token.head.pos_,[child for child in token.children])
		if token.dep_ == "ROOT":
			root = token.text
	return out, root

# NER tagging
def ner_tag(text):
	NER_tag_dict = dict()
	for i,line in enumerate(text):
		tags = []
		doc = nlp(str(line))

		for ent in doc.ents:
			# print(ent.text +'-' + ent.label_ + '\n')
			tags.append(ent.text +'-' + ent.label_)
		if len(tags) != 0:
			NER_tag_dict[i] = tags
	return NER_tag_dict

def ner_tag_sentence(sentence):
	doc = nlp(str(sentence))
	NER_tag_dict = dict()
	tags = []
	for ent in doc.ents:
		# print(ent.text +'-' + ent.label_ + '\n')
		NER_tag_dict[ent.text] = ent.label_
	return NER_tag_dict


#check tense of verb
def check_tense(root, pos_dict):
	tag = pos_dict[root][1]
	if tag == "VB":
		return "do"
	elif tag == "VBD":
		return "did"
	elif tag == "VBG":
		return "doing"
	elif tag == "VBN":
		return "done"
	elif tag == "VBP":
		return "do"
	elif tag == "VBZ":
		return "does"
	else:
		return None

# return dictionary of lemmas in sentence
def getTokenLemma(nlp_doc):
	lemmas = {}
	for token in nlp_doc:
		lemmas[str(token)] = token.lemma_
	return lemmas


TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-3-deafc25b4eb5>, line 7)

In [3]:
# return dictionary of lemmas in sentence
def getTokenLemma(nlp_doc):
	lemmas = {}
	for token in nlp_doc:
		lemmas[str(token)] = token.lemma_
	return lemmas

#check tense of verb
def check_tense(root, pos_dict):
	tag = pos_dict[root][1]
	if tag == "VB":
		return "do"
	elif tag == "VBD":
		return "did"
	elif tag == "VBG":
		return "doing"
	elif tag == "VBN":
		return "done"
	elif tag == "VBP":
		return "do"
	elif tag == "VBZ":
		return "does"
	else:
		return None

In [4]:
# def pos_tag_sentence(sentence):
#     #list of sentences
#     POS_tag_dict = dict()
#     text = sentence.split()
#     for i,line in enumerate(text):
#         tags = []
#         doc = nlp(str(line))
#         for token in doc:
#             tags.append((token.text, token.pos_, token.tag_, token.dep_, token.is_stop, ))
#         if len(tags) != 0:
#             POS_tag_dict[i] = tags
#     return POS_tag_dict
def pos_tag_sentence(sentence):
	# list of sentences
	POS_tag_dict = dict()
	text = sentence.split()
	for line in text:
		tags = []
		doc = nlp(str(line))
		for token in doc:
			tags.append((token.pos_, token.tag_, token.dep_, token.is_stop,))
		if len(tags) != 0:
			POS_tag_dict[token.text] = tags[0]
	return POS_tag_dict

def pos_tag_lst(text):
    #list of sentences
    POS_tag_dict = dict()
    for i,line in enumerate(text):
        tags = []
        doc = nlp(str(line))
        for token in doc:
            tags.append((token.text, token.pos_, token.tag_, token.dep_, token.is_stop))
        if len(tags) != 0:
            POS_tag_dict[i] = tags
    return POS_tag_dict

#Token dict 
def dependency_dict(doc):
    out = dict()
    root = ''
    for token in doc:
        out[token.text] = (token.dep_, token.head.text, token.head.pos_,[child for child in token.children])
        if token.dep_ == "ROOT":
            root = token.text
    return out, root

def ner_tag_sentence(sentence):
    doc = nlp(str(sentence))
    NER_tag_dict = dict()
    tags = []
    for ent in doc.ents:
        # print(ent.text +'-' + ent.label_ + '\n')
        NER_tag_dict[ent.text] = ent.label_
    return NER_tag_dict



In [22]:
def HowOftenQ(sentence, ner_tag_dict, dependency_dict, root):
    date = ""
    output = ""
    clause = ""
    sentence = sentence[:-1]
    for k in ner_tag_dict.keys():
        if ner_tag_dict[k] == 'DATE':
            date = k
    if date == "": return
    date_lst = date.split()
    sentence_lst = sentence.split()
    root_lst_ind = sentence_lst.index(root)
    root_ind = sentence.index(root)
    date_ind = sentence.index(date)
    if date_ind < root_ind:
        word_in_front_of_root = sentence_lst[root_lst_ind - 1]
        if token_Dict[word_in_front_of_root][0] == 'auxpass':
            sentence_lst[root_lst_ind - 1] = sentence_lst[root_lst_ind - 2]
            sentence_lst[root_lst_ind - 2] = word_in_front_of_root
        else:
            sentence_lst[root_lst_ind] = nlp(root)[0].lemma_
            if pos_tag_sentence(root)[0][0][2] in ("VBD", "VBN"):
                sentence_lst.insert(root_lst_ind-1,"did")
            else: sentence_lst.insert(root_lst_ind-1,"does")
        for i in date_lst:
            sentence_lst.remove(i)
        output = "How long " + " ".join(sentence_lst)
    else:
        for seg in sentence.split(',', 1):
            if date in seg: clause = seg
        if root_ind != 0:
            word_in_front_of_root = sentence_lst[root_lst_ind - 1] 
            word_after_root = sentence_lst[root_lst_ind + 1]
            prep_ind = sentence.index(word_after_root)
            if token_Dict[word_after_root][0] != 'prep': word_after_root = ""
            #if it's passive tense
            if token_Dict[word_in_front_of_root][0] == 'auxpass':
                output = "How long " + word_in_front_of_root + " " + clause.split(word_in_front_of_root,1)[0].lower() +  sentence[root_ind:prep_ind+len(word_after_root)]
            else:
                #if it's past tense
                if pos_tag_sentence(root)[0][0][2] in ("VBD", "VBN"):
                    output = "How long" + " did " + clause.split(root,1)[0].lower() + nlp(root)[0].lemma_ + sentence[root_ind+len(root):prep_ind+len(word_after_root)]
                #if it's not past tense
                else:
                    output = "How long" + " does " + clause.split(root,1)[0].lower() + nlp(root)[0].lemma_ + sentence[root_ind+len(root):prep_ind+len(word_after_root)]
    output = ' '.join(output.split()) + "?"
    return output

In [32]:
sentence = "Four months after Harris's death, Cuarón chose Gambon as his replacement."
#sentence = "With Prisoner of Azkaban, production of the Harry Potter films was switched to an eighteen-month cycle."
# sentence = "Harry Potter and the Prisoner of Azkaban is a 2004 fantasy film directed by Alfonso Cuaron and distributed by Warner Bros. Pictures."
doc = nlp(sentence)
token_Dict, root = dependency_dict(doc)
ner_tag_dict = ner_tag_sentence(sentence)
pos_tag_dict = pos_tag_sentence(sentence)

In [36]:
pos_tag_dict

{'Four': ('NUM', 'CD', 'ROOT', True),
 'months': ('NOUN', 'NNS', 'ROOT', False),
 'after': ('ADP', 'IN', 'ROOT', True),
 "'s": ('PROPN', 'NNP', 'ROOT', False),
 ',': ('NOUN', 'NN', 'ROOT', False),
 'Cuarón': ('PROPN', 'NNP', 'ROOT', False),
 'chose': ('VERB', 'VBD', 'ROOT', False),
 'Gambon': ('PROPN', 'NNP', 'ROOT', False),
 'as': ('ADP', 'IN', 'ROOT', True),
 'his': ('PRON', 'PRP$', 'ROOT', True),
 '.': ('NOUN', 'NN', 'ROOT', False)}

In [24]:
pos_tag_sentence(root)

{0: [('is', 'AUX', 'VBZ', 'ROOT', True)]}

In [26]:
pos_tag_sentence(sentence)

{0: [('Four', 'NUM', 'CD', 'ROOT', True)],
 1: [('months', 'NOUN', 'NNS', 'ROOT', False)],
 2: [('after', 'ADP', 'IN', 'ROOT', True)],
 3: [('Harris', 'PROPN', 'NNP', 'ROOT', False),
  ("'s", 'PART', 'POS', 'case', True)],
 4: [('death', 'NOUN', 'NN', 'ROOT', False),
  (',', 'PUNCT', ',', 'punct', False)],
 5: [('Cuarón', 'PROPN', 'NNP', 'ROOT', False)],
 6: [('chose', 'VERB', 'VBD', 'ROOT', False)],
 7: [('Gambon', 'PROPN', 'NNP', 'ROOT', False)],
 8: [('as', 'ADP', 'IN', 'ROOT', True)],
 9: [('his', 'PRON', 'PRP$', 'ROOT', True)],
 10: [('replacement', 'NOUN', 'NN', 'ROOT', False),
  ('.', 'PUNCT', '.', 'punct', False)]}

In [23]:
HowOftenQ(sentence, ner_tag_dict, token_Dict, root)

'How long does harry potter and the prbe?'

In [19]:
def whereQ(sentence, dep_dict, root):
    output = ''
    verbs = ['was', 'is', 'were']
    foundVerb = False
    foundVerbInd = 0
    # find tense 
    pos_tags = pos_tag_sentence(sentence)
    seenWhere = False
    foundWhereInd = 0
    whereInd = 0
    tense = None
    for word in sentence.split():
        if word == 'where':
            seenWhere = True
            foundWhereInd = whereInd
            verb = dep_dict[word][1]
            if pos_tags[verb][0] == 'VERB':
                tense = check_tense(verb, pos_tags)
            elif verb in verbs: # if tense is was, is, were
                tense = verb
                foundVerb = True
                foundVerbInd = sentence.split().index(verb)
            break
        whereInd += 1
    if tense == None: # not able to create question from sentence
        return "No question"
    output += f'Where {tense} '
    ind = foundWhereInd
    if foundVerb:
        ind = foundVerbInd
    for word in sentence.split()[ind+1:]:
        output += word + " "    
    return output[:-2] + "?"    

In [7]:
sentence = "This leads the trio to an underground passage to the Shrieking Shack, where they discover that the dog is actually Sirius, who is an Animagus."
dep_dict1, root1 = dependency_dict(nlp(sentence))
print(dep_dict1, root1)

{'This': ('nsubj', 'leads', 'VERB', []), 'leads': ('ROOT', 'leads', 'VERB', [This, trio, to, .]), 'the': ('det', 'dog', 'NOUN', []), 'trio': ('dobj', 'leads', 'VERB', [the]), 'to': ('prep', 'passage', 'NOUN', [Shack]), 'an': ('det', 'Animagus', 'PROPN', []), 'underground': ('amod', 'passage', 'NOUN', []), 'passage': ('pobj', 'to', 'ADP', [an, underground, to]), 'Shrieking': ('compound', 'Shack', 'PROPN', []), 'Shack': ('pobj', 'to', 'ADP', [the, Shrieking, ,, discover]), ',': ('punct', 'Sirius', 'PROPN', []), 'where': ('advmod', 'discover', 'VERB', []), 'they': ('nsubj', 'discover', 'VERB', []), 'discover': ('relcl', 'Shack', 'PROPN', [where, they, is]), 'that': ('mark', 'is', 'AUX', []), 'dog': ('nsubj', 'is', 'AUX', [the]), 'is': ('relcl', 'Sirius', 'PROPN', [who, Animagus]), 'actually': ('advmod', 'is', 'AUX', []), 'Sirius': ('attr', 'is', 'AUX', [,, is]), 'who': ('nsubj', 'is', 'VERB', []), 'Animagus': ('attr', 'is', 'VERB', [an]), '.': ('punct', 'leads', 'VERB', [])} leads


In [10]:
sentence2 = "He was in the underground, where there were lots of tea."
dep_dict2, root2 = dependency_dict(nlp(sentence2))
print(dep_dict2, root2)

{'He': ('nsubj', 'was', 'AUX', []), 'was': ('ROOT', 'was', 'AUX', [He, in, .]), 'in': ('prep', 'was', 'AUX', [underground]), 'the': ('det', 'underground', 'NOUN', []), 'underground': ('pobj', 'in', 'ADP', [the, ,, were]), ',': ('punct', 'underground', 'NOUN', []), 'where': ('advmod', 'were', 'VERB', []), 'there': ('expl', 'were', 'VERB', []), 'were': ('relcl', 'underground', 'NOUN', [where, there, lots]), 'lots': ('attr', 'were', 'VERB', [of]), 'of': ('prep', 'lots', 'NOUN', [tea]), 'tea': ('pobj', 'of', 'ADP', []), '.': ('punct', 'was', 'AUX', [])} was


In [11]:
pos_tag_sentence(sentence2)

{'He': ('PRON', 'PRP', 'ROOT', True),
 'was': ('AUX', 'VBD', 'ROOT', True),
 'in': ('ADP', 'IN', 'ROOT', True),
 'the': ('DET', 'DT', 'ROOT', True),
 ',': ('ADV', 'RB', 'ROOT', False),
 'where': ('ADV', 'WRB', 'ROOT', True),
 'there': ('ADV', 'RB', 'ROOT', True),
 'were': ('AUX', 'VBD', 'ROOT', True),
 'lots': ('NOUN', 'NNS', 'ROOT', False),
 'of': ('ADP', 'IN', 'ROOT', True),
 '.': ('NOUN', 'NN', 'ROOT', False)}

In [9]:
pos_tag_sentence(sentence)

{'This': ('DET', 'DT', 'ROOT', True),
 'leads': ('VERB', 'VBZ', 'ROOT', False),
 'the': ('DET', 'DT', 'ROOT', True),
 'trio': ('NOUN', 'NN', 'ROOT', False),
 'to': ('PART', 'TO', 'ROOT', True),
 'an': ('DET', 'DT', 'ROOT', True),
 'underground': ('ADV', 'RB', 'ROOT', False),
 'passage': ('NOUN', 'NN', 'ROOT', False),
 'Shrieking': ('VERB', 'VBG', 'ROOT', False),
 ',': ('PROPN', 'NNP', 'ROOT', False),
 'where': ('ADV', 'WRB', 'ROOT', True),
 'they': ('PRON', 'PRP', 'ROOT', True),
 'discover': ('VERB', 'VB', 'ROOT', False),
 'that': ('DET', 'DT', 'ROOT', True),
 'dog': ('NOUN', 'NN', 'ROOT', False),
 'is': ('AUX', 'VBZ', 'ROOT', True),
 'actually': ('ADV', 'RB', 'ROOT', False),
 'who': ('PRON', 'WP', 'ROOT', True),
 '.': ('PROPN', 'NNP', 'ROOT', False)}

In [15]:
print(whereQ(sentence, dep_dict1, root1))


Where do they discover that the dog is actually Sirius, who is an Animagus?


In [20]:
print(whereQ(sentence2, dep_dict2, root2))

Where were lots of tea?


# Question Asking- Checking Type

In [16]:
#check if the sentence contains certain NER tags
#When - DATE
#Who - PERSON
#Where" - FAC, ORG, ORG, ORG, GPE
#"How many" - CARDINAL
#"How long" - DATE
#"How much" - MONEY
#"Why" - "because"

def check_NER(sentence, question_type):
    output = False
    ner_tag_dict = ner_tag_sentence(sentence)
    if question_type == "When":
        if ("TIME" in ner_tag_dict.values()) or ("DATE" in ner_tag_dict.values()):
            output = True
    elif question_type == "Who":
        if ("PERSON" in ner_tag_dict.values()):
            output = True
    elif question_type == "Where":  
        if ("FAC" in ner_tag_dict.values()) or ("ORG" in ner_tag_dict.values()) or ("LOC" in ner_tag_dict.values()):
            output = True
    elif question_type == "How much":
        if ("MONEY" in ner_tag_dict.values()):
            output = True
    elif question_type == "How long":
        if ("DATE" in ner_tag_dict.values()):
            output = True
    elif question_type == "How many":
        if ("CARDINAL" in ner_tag_dict.values()):
            output = True
    elif question_type == "How often":
        if ("CARDINAL" in ner_tag_dict.values()):
            output = True
    elif question_type == "Why":
        if ("because" in sentence) or ("due to" in sentence) or ("Due to" in sentence):
            output = True
    else:
        for aux in auxiliary_verbs:
            aux_cap = aux[0].upper() + aux[1:]
            if aux or aux_cap in sentence:
                output = True
    return output

def checkSentenceType(sentence, dep_dict, ner_tag_dict, root):
    possible_types = set()
    if ("TIME" in ner_tag_dict.values()) or ("DATE" in ner_tag_dict.values()):
        possible_types.add("When")
    if ("PERSON" in ner_tag_dict.values()):
        possible_types.add("Who")
    if ("FAC" in ner_tag_dict.values()) or ("ORG" in ner_tag_dict.values()) or ("LOC" in ner_tag_dict.values()) or ("GPE" in ner_tag_dict.values()):
        possible_types.add("Where")
    if ("MONEY" in ner_tag_dict.values()):
        possible_types.add("How much")
    if ("DATE" in ner_tag_dict.values()):
        possible_types.add("How long")
    if ("CARDINAL" in ner_tag_dict.values()):
        possible_types.add("How many")
        possible_types.add("How often")
    if ("because" in sentence) or ("due to" in sentence) or ("Due to" in sentence) or ("since" in sentence):
        possible_types.add("Why")
    aux_verbs = {"are", "is", "was", "were", "shall", "do", "does", "did", 
                 "can", "could", "have", "need", "should", "will", "would",
                 "must", "may", "might", "cannot"}
    if root in aux_verbs:
        possible_types.add("What")
    for dep in dep_dict.values():
        if dep[0] == 'nsubj':
            possible_types.add("What")
    return possible_types
    

In [37]:
sentence = "It is based on the novel of the same name by J. K. Rowling."
# sentence = "The story takes place in Hollywood, between 1927 and 1932, and focuses on the relationship of an older silent film star and a rising young actress as silent cinema falls out of fashion and is replaced by the 'talkies'."
# sentence = "The film, which is the third instalment in the Harry Potter film series, was written by Steve Kloves and produced by Chris Columbus (director of the first two instalments), David Heyman, and Mark Radcliffe."
# sentence = "The film stars Daniel Radcliffe as Harry Potter, alongside Rupert Grint and Emma Watson as Harry's best friends Ron Weasley and Hermione Granger."
sentence = "However, it was, at the time, the best-reviewed film of the series and currently the second-best-reviewed film of the franchise according to review aggregator Rotten Tomatoes."
nlp_doc = nlp(sentence)
dep_dict, root = dependency_dict(nlp_doc)
ner_tag_dict = ner_tag_sentence(sentence)


In [38]:
dep_dict

{'However': ('advmod', 'was', 'AUX', []),
 ',': ('punct', 'was', 'AUX', []),
 'it': ('nsubj', 'was', 'AUX', []),
 'was': ('ROOT', 'was', 'AUX', [However, ,, it, ,, at, ,, film, .]),
 'at': ('prep', 'was', 'AUX', [time]),
 'the': ('det', 'franchise', 'NOUN', []),
 'time': ('pobj', 'at', 'ADP', [the]),
 'best': ('advmod', 'reviewed', 'VERB', []),
 '-': ('punct', 'reviewed', 'VERB', []),
 'reviewed': ('amod', 'film', 'NOUN', [second, -, best, -]),
 'film': ('conj', 'film', 'NOUN', [currently, the, reviewed, of]),
 'of': ('prep', 'film', 'NOUN', [franchise]),
 'series': ('pobj', 'of', 'ADP', [the]),
 'and': ('cc', 'film', 'NOUN', []),
 'currently': ('advmod', 'film', 'NOUN', []),
 'second': ('advmod', 'reviewed', 'VERB', []),
 'franchise': ('pobj', 'of', 'ADP', [the]),
 'according': ('prep', 'film', 'NOUN', [review]),
 'to': ('aux', 'review', 'NOUN', []),
 'review': ('pcomp', 'according', 'VERB', [to, aggregator]),
 'aggregator': ('dobj', 'review', 'NOUN', []),
 'Rotten': ('compound', 'Tom

In [39]:
ner_tag_dict

{'second': 'ORDINAL', 'Rotten Tomatoes': 'PERSON'}

In [40]:
checkSentenceType(sentence, dep_dict, ner_tag_dict, root)

{'What', 'Who'}

In [20]:
class GenerateQuestions(object):

	def __init__(self):
		textfile = "test.txt"
		self.nlp = spacy.load('en_core_web_sm')
		self.asker = Asking.Asking(textfile)
		self.parser = Parser.Parser(textfile)

	def checkSentenceType(self, sentence, dep_dict, ner_tag_dict, root):
		possible_types = set()
		if ("TIME" in ner_tag_dict.values()) or (
				"DATE" in ner_tag_dict.values()):
			possible_types.add("When")
		if ("PERSON" in ner_tag_dict.values()):
			possible_types.add("Who")
		if ("FAC" in ner_tag_dict.values()) or (
				"ORG" in ner_tag_dict.values()) or (
				"LOC" in ner_tag_dict.values()) or (
				"GPE" in ner_tag_dict.values()):
			possible_types.add("Where")
		if ("MONEY" in ner_tag_dict.values()):
			possible_types.add("How much")
		if ("DATE" in ner_tag_dict.values()):
			possible_types.add("How long")
		if ("CARDINAL" in ner_tag_dict.values()):
			possible_types.add("How many")
			possible_types.add("How often")
		if ("because" in sentence) or ("due to" in sentence) or (
				"Due to" in sentence) or ("since" in sentence):
			possible_types.add("Why")
		aux_verbs = {"are", "is", "was", "were", "shall", "do", "does", "did",
					 "can", "could", "have", "need", "should", "will", "would",
					 "must", "may", "might", "cannot"}
		if root in aux_verbs:
			possible_types.add("What")
		for dep in dep_dict.values():
			if dep[0] == 'nsubj':
				possible_types.add("What")
		return possible_types

	def generateQuestions(self, limit):
		possible_questions = set()
		for sentence in self.parser.text:
			print(sentence)
			nlp_doc = self.nlp(sentence)
			pos_tags = self.parser.pos_tag_sentence(sentence)
			token_dict, root = self.parser.dependency_dict(nlp_doc)
			ner_tags = self.parser.ner_tag_sentence(sentence)
			lemma_dict = self.parser.getTokenLemma(nlp_doc)
			type_dict = {"How many": self.asker.howManyQ(sentence, ner_tags,
												token_dict, pos_tags, root),
						 "Who": self.asker.whoQ(sentence, ner_tags, token_dict),
						 "How much": self.asker.howMuchQ(sentence, nlp_doc, ner_tags,
													  token_dict, root, pos_tags),
						 "How often": self.asker.howOftenQ(sentence, ner_tags, token_dict,
														pos_tags, root),
						 "Why": self.asker.whyQ(sentence, nlp_doc, ner_tags, token_dict,
											 pos_tags, root),
						 "Where": self.asker.whereQ(sentence, token_dict, pos_tags),
						 "What": self.asker.whatQ(sentence, token_dict, root, pos_tags,
											   lemma_dict),
						 "When": self.asker.whenQ(sentence, ner_tags, root, nlp_doc,
											   pos_tags)}

			# generate question outputs
			try:
				possible_types = self.checkSentenceType(sentence, token_dict,
														ner_tags, root)
				for type in possible_types:
					question = type_dict[type]
					if self.isValidQuestion(question):
						print(question)
						possible_questions.add(question)
					binary_output = self.asker.binaryQ(sentence, root)
				if self.isValidQuestion(binary_output):
					print(binary_output)
					possible_questions.add(binary_output)
			except:
				continue

			# limit amount of questions to be generated
			if len(possible_questions) >= limit:
				return possible_questions
		return possible_questions

	# check if generated question is valid
	def isValidQuestion(self, question):
		return question != None and len(question) > 0

In [ ]:
generator = GenerateQuestions()
generator.generateQuestions(50)

# Who Question Asking Revision

In [82]:
# input: a single sentence, and its ner tag dict and dependency dict
# Who Question
def whoQ(sentence, ner_tag_dict, dependency_dict, root):
    # find PERSON tag
    theName = ''
    output = ''
    if root not in sentence:
        return
    rootInd = sentence.index(root)
    minDist = len(sentence)
    for ner_tag in ner_tag_dict:
        if ner_tag_dict[ner_tag] == 'PERSON':
            tag_ind = sentence.index(ner_tag)
            if abs(tag_ind - rootInd) < minDist:
                minDist = abs(tag_ind - rootInd)
                theName = ner_tag
    if theName == '':
        return
    if "'" in theName:
        output = sentence.replace(theName, 'whose', 1)
    else: 
        output = sentence.replace(theName, 'who', 1)
    output = output[:-1] + "?"
    output = output[0].upper() + output[1:]
    return output

In [75]:
sentence = "The story follows Harry Potter's third year at Hogwarts as he is informed that a prisoner named Sirius Black has escaped from Azkaban intending to kill him."
nlp_doc = nlp(sentence)
dep_dict, root = dependency_dict(nlp_doc)
ner_tag_dict = ner_tag_sentence(sentence)

In [84]:
whoQ(sentence, ner_tag_dict, dep_dict, root)

'The story follows whose third year at Hogwarts as he is informed that a prisoner named Sirius Black has escaped from Azkaban intending to kill him?'

In [87]:
sentence2 = "The film stars Daniel Radcliffe as Harry Potter, alongside Rupert Grint and Emma Watson as Harry's best friends Ron Weasley and Hermione Granger."
# sentence2 = "It also features a star-studded supporting cast including Gary Oldman, David Thewlis, Michael Gambon (in his debut role as Albus Dumbledore), Emma Thompson and Timothy Spall."
# sentence2 = "It is the sequel to Harry Potter and the Chamber of Secrets and is followed by Harry Potter and the Goblet of Fire."
nlp_doc2 = nlp(sentence2)
dep_dict2, root2 = dependency_dict(nlp_doc2)
ner_tag_dict2 = ner_tag_sentence(sentence2)

In [88]:
whoQ(sentence2, ner_tag_dict2, dep_dict2, root2)

"The film stars who as Harry Potter, alongside Rupert Grint and Emma Watson as Harry's best friends Ron Weasley and Hermione Granger?"

# Binary Asking Revision

In [212]:
# sentence = "The film was nominated for two Academy Awards, Best Original Music Score and Best Visual Effects at the 77th Academy Awards in 2005."
# sentence = "It is also the last Harry Potter film to be released on VHS as well as the last film until Harry Potter and the Half-Blood Prince to be rated PG in North America."
# sentence = "Prisoner of Azkaban grossed a total of $796.7 million worldwide, with its box office performance ranking as the lowest-grossing in the series."
# sentence = "However, it was, at the time, the best-reviewed film of the series and currently the second-best-reviewed film of the franchise according to review aggregator Rotten Tomatoes."
sentence = "Harry Potter and the Prisoner of Azkaban is a 2004 fantasy film directed by Alfonso Cuar�n and distributed by Warner Bros. Pictures."
nlp_doc = nlp(sentence)
dep_dict, root = dependency_dict(nlp_doc)
ner_tag_dict = ner_tag_sentence(sentence)
pos_tag_dict = pos_tag_sentence(sentence)

In [183]:
print(getTokenLemma(nlp_doc))

{'Prisoner': 'Prisoner', 'of': 'of', 'Azkaban': 'Azkaban', 'grossed': 'gross', 'a': 'a', 'total': 'total', '$': '$', '796.7': '796.7', 'million': 'million', 'worldwide': 'worldwide', ',': ',', 'with': 'with', 'its': 'its', 'box': 'box', 'office': 'office', 'performance': 'performance', 'ranking': 'rank', 'as': 'as', 'the': 'the', 'lowest': 'lowest', '-': '-', 'grossing': 'gross', 'in': 'in', 'series': 'series', '.': '.'}


In [182]:
pos_tags

{'Prisoner': ('PROPN', 'NNP', 'ROOT', False),
 'of': ('ADP', 'IN', 'ROOT', True),
 'Azkaban': ('ADJ', 'JJ', 'ROOT', False),
 'grossed': ('VERB', 'VBN', 'ROOT', False),
 'a': ('DET', 'DT', 'ROOT', True),
 'total': ('ADJ', 'JJ', 'ROOT', False),
 '796.7': ('SYM', '$', 'nmod', False),
 'million': ('NUM', 'CD', 'ROOT', False),
 ',': ('ADV', 'RB', 'ROOT', False),
 'with': ('ADP', 'IN', 'ROOT', True),
 'its': ('PRON', 'PRP$', 'ROOT', True),
 'box': ('NOUN', 'NN', 'ROOT', False),
 'office': ('NOUN', 'NN', 'ROOT', False),
 'performance': ('NOUN', 'NN', 'ROOT', False),
 'ranking': ('VERB', 'VBG', 'ROOT', False),
 'as': ('ADP', 'IN', 'ROOT', True),
 'the': ('DET', 'DT', 'ROOT', True),
 'grossing': ('ADV', 'RBS', 'advmod', False),
 'in': ('ADP', 'IN', 'ROOT', True),
 '.': ('PROPN', 'NNP', 'ROOT', False)}

In [168]:
dep_dict

{'Prisoner': ('nsubj', 'grossed', 'VERB', [of]),
 'of': ('prep', 'total', 'NOUN', [million]),
 'Azkaban': ('pobj', 'of', 'ADP', []),
 'grossed': ('ROOT',
  'grossed',
  'VERB',
  [Prisoner, total, worldwide, ,, with, .]),
 'a': ('det', 'total', 'NOUN', []),
 'total': ('dobj', 'grossed', 'VERB', [a, of]),
 '$': ('quantmod', 'million', 'NUM', []),
 '796.7': ('compound', 'million', 'NUM', []),
 'million': ('pobj', 'of', 'ADP', [$, 796.7]),
 'worldwide': ('advmod', 'grossed', 'VERB', []),
 ',': ('punct', 'grossed', 'VERB', []),
 'with': ('prep', 'grossed', 'VERB', [ranking]),
 'its': ('poss', 'performance', 'NOUN', []),
 'box': ('compound', 'office', 'NOUN', []),
 'office': ('compound', 'performance', 'NOUN', [box]),
 'performance': ('nsubj', 'ranking', 'VERB', [its, office]),
 'ranking': ('pcomp', 'with', 'ADP', [performance, as]),
 'as': ('prep', 'ranking', 'VERB', [grossing]),
 'the': ('det', 'series', 'NOUN', []),
 'lowest': ('advmod', 'grossing', 'VERB', []),
 '-': ('punct', 'grossing

In [204]:
# input: a single sentence, with its dependency dict and root word
auxiliary_verbs = ["am", "is", "are", "was", "were", "shall", "do", "does", "did","can", "could", "have", "need",
                   "should", "will", "would"]
def binaryQ(sentence, dep_dict, ner_tag_dict, pos_tag_dict, nlp_doc, root):
    output = ''
    seenRoot = False
    if root not in sentence:
        return
    split = sentence.split()
    if root + "," in split:
        rootInd = split.index(root + ',')
    else:
        rootInd = split.index(root)
    ner_tags = set()
    for ner_tag in ner_tag_dict:
        split_tag = ner_tag.split()
        for split_word in split_tag:
            ner_tags.add(split_word)
    if root in auxiliary_verbs:
        output += root.capitalize() + ' '
        for word in split[0:rootInd]:
            if word not in ner_tags:
                output += word.lower() + ' '
            else:
                output += word + ' '
        output += " ".join(split[rootInd + 1:])
        output = output[:-1] + '?'
        return output
    else:
        aux_verb = ""
        minDist = len(split)
        for word in split:
            if word in auxiliary_verbs:
                if abs(split.index(word) - rootInd) < minDist:
                    minDist = split.index(word) - rootInd
                    aux_verb = word
        if aux_verb != "":
            output += aux_verb.capitalize() + ' '
            auxInd = split.index(aux_verb)
            output += " ".join(split[0:auxInd]) # add everything before aux verb
            output += " " + root + " "
            output += " ".join(split[rootInd + 1:])
        else:
            # edge case where root and nearby words not in auxiliary verbs
            tense = check_tense(root, pos_tag_dict)
            if tense == None: return
            output += tense.capitalize() + ' '
            output += " ".join(split[0:rootInd]) + ' '
            output += getTokenLemma(nlp_doc)[root] + ' '
            output += " ".join(split[rootInd + 1:])
        split_output = output.split()
        if split_output[1] not in ner_tags:
            output = split_output[0] + ' ' + split_output[1].lower() + ' ' + " ".join(split_output[2:])
        output = output[:-1] + '?'
        return output

In [213]:
binaryQ(sentence, dep_dict, ner_tag_dict, pos_tag_dict, nlp_doc, root)

'Is Harry Potter and the Prisoner of Azkaban a 2004 fantasy film directed by Alfonso Cuar�n and distributed by Warner Bros. Pictures?'

# What Question Revision

In [66]:
def whatQ(sentence, dep_dict, root):
    aux_verbs = {"are", "is", "was", "were", "shall", "do", "does", "did", 
                 "can", "could", "have", "need", "should", "will", "would", 
                 "must", "may", "might", "cannot"}
    output = ''
    if root in aux_verbs:
        output += f"What {root} "
        seenRoot = False
        for n in sentence.split():
            if n == root or n == root + ',':
                seenRoot = True
                continue
            if seenRoot:
                output += n + " "
        return output[:-2] + "?"
    else:
        splitSentence = sentence.split()
        if root in splitSentence:
            rootInd = splitSentence.index(root)
        elif root + ',' in splitSentence:
            rootInd = splitSentence.index(root + ',')
        else:
            return
        if rootInd != 0:
            if splitSentence[rootInd - 1] in aux_verbs:
                output += f'What {splitSentence[rootInd - 1]} {root} '
                output += " ".join(splitSentence[rootInd + 1:])
                return output[:-1] + "?"
            elif rootInd >= 2 and splitSentence[rootInd - 1] == 'be' and splitSentence[rootInd - 2] in aux_verbs:
                output += 'What '
                output += " ".join(splitSentence[rootInd - 2:])
                return output[:-1] + "?"
            else:
                # case where nsubj is before the root
                for word in splitSentence[:rootInd + 1]:
                    if word in dep_dict and dep_dict[word][0] == 'nsubj':
                        output += 'What '
                        output += " ".join(splitSentence[rootInd:])
                        return output[:-1] + "?"
        return        

In [73]:
sentence = "The film stars Daniel Radcliffe as Harry Potter, alongside Rupert Grint and Emma Watson as Harry's best friends Ron Weasley and Hermione Granger."
sentence = "Prisoner of Azkaban grossed a total of $796.7 million worldwide, with its box office performance ranking as the lowest-grossing in the series."
sentence = "The film would be nominated for two Academy Awards, Best Original Music Score and Best Visual Effects at the 77th Academy Awards in 2005."
# sentence = "However, it was, at the time, the best-reviewed film of the series and currently the second-best-reviewed film of the franchise according to review aggregator Rotten Tomatoes."
# sentence = "The film was released on 31 May 2004 in the United Kingdom and on 4 June 2004 in North America, as the first Harry Potter film released into IMAX theatres and to be using IMAX Technology."
nlp_doc = nlp(sentence)
dep_dict, root = dependency_dict(nlp_doc)
ner_tag_dict = ner_tag_sentence(sentence)
pos_tag_dict = pos_tag_sentence(sentence)
lemmas = getTokenLemma(nlp_doc)

In [74]:
dep_dict

{'The': ('det', 'film', 'NOUN', []),
 'film': ('nsubjpass', 'nominated', 'VERB', [The]),
 'would': ('aux', 'nominated', 'VERB', []),
 'be': ('auxpass', 'nominated', 'VERB', []),
 'nominated': ('ROOT', 'nominated', 'VERB', [film, would, be, for, in, .]),
 'for': ('prep', 'nominated', 'VERB', [Awards]),
 'two': ('nummod', 'Awards', 'PROPN', []),
 'Academy': ('compound', 'Awards', 'PROPN', []),
 'Awards': ('pobj', 'at', 'ADP', [the, 77th, Academy]),
 ',': ('punct', 'Awards', 'PROPN', []),
 'Best': ('conj', 'Score', 'PROPN', []),
 'Original': ('compound', 'Score', 'PROPN', []),
 'Music': ('compound', 'Score', 'PROPN', []),
 'Score': ('nmod', 'Effects', 'NOUN', [Best, Original, Music, and, Best]),
 'and': ('cc', 'Score', 'PROPN', []),
 'Visual': ('compound', 'Effects', 'NOUN', []),
 'Effects': ('appos', 'Awards', 'PROPN', [Score, Visual, at]),
 'at': ('prep', 'Effects', 'NOUN', [Awards]),
 'the': ('det', 'Awards', 'PROPN', []),
 '77th': ('compound', 'Awards', 'PROPN', []),
 'in': ('prep', '

In [ ]:
# cases:
# Q: What grossed a total of $796.7 million worldwide....? root pos = VBN
# Q: What stars Daniel Radcliffe as Harry Potter, alongside ....?
# Q: What was nominated for two Academy Awards,...? root pos = VBN
# Q: What was released on 31 May 2004...?
# Process:
    # What + root + everything after
    # if auxiliary verb is right before root, add auxiliary verb in front of root
    # check if there's a 'be' before root and auxiliary verb before 'be' 
        #(if so, it should be What + aux verb + 'be' + root + everything else)


In [75]:
whatQ(sentence, dep_dict, root)

'What would be nominated for two Academy Awards, Best Original Music Score and Best Visual Effects at the 77th Academy Awards in 2005?'